In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid

In [2]:
# Set parameters for linear regression

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = [1.0, 'sqrt', 'log2']
# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
#max_depth.append(None)
# Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

warm_start = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               #'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'warm_start': warm_start}

grid = ParameterGrid(random_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)


Total Possible Models 360


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_30min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

data['hour_of_day'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_month'] = data['date'].dt.day
data['month_of_year'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# make a weekend column
data['weekend'] = 0
data.loc[data['day_of_week'] == 5, 'weekend'] = 1
data.loc[data['day_of_week'] == 6, 'weekend'] = 1

#make a monday column
data['monday'] = 0
data.loc[data['day_of_week'] == 0, 'monday'] = 1

#make a tuesday column
data['tuesday'] = 0
data.loc[data['day_of_week'] == 1, 'tuesday'] = 1

#make a wednesday column
data['wednesday'] = 0
data.loc[data['day_of_week'] == 2, 'wednesday'] = 1

#make a thursday column
data['thursday'] = 0
data.loc[data['day_of_week'] == 3, 'thursday'] = 1

#make a friday column
data['friday'] = 0
data.loc[data['day_of_week'] == 4, 'friday'] = 1

#make a saturday column
data['saturday'] = 0
data.loc[data['day_of_week'] == 5, 'saturday'] = 1

#make a sunday column
data['sunday'] = 0
data.loc[data['day_of_week'] == 6, 'sunday'] = 1


# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Define the feature columns and the target column
feature_cols = ['time', 'hour_of_day', 'day_of_week', 'day_of_month', 'month_of_year', 'year', 'weekend', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
target_col = 'y'

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = RandomForestRegressor()

search = RandomizedSearchCV(estimator = model, 
                            param_distributions = random_grid, 
                            n_iter = 25, 
                            cv = tscv,
                            refit=True, 
                            verbose=3, 
                            random_state=42, 
                            n_jobs = -1, 
                            scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                            error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[CV 1/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=64, warm_start=True;, score=-0.877 total time=   0.4s
[CV 2/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=64, warm_start=True;, score=-0.913 total time=   0.7s
[CV 3/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=64, warm_start=True;, score=-0.920 total time=   1.2s
[CV 1/5] END bootstrap=False, max_features=1.0, min_samples_leaf=4, n_estimators=118, warm_start=True;, score=-0.982 total time=   1.2s
[CV 1/5] END bootstrap=False, max_features=sqrt, min_samples_leaf=4, n_estimators=118, warm_start=False;, score=-1.152 total time=   0.3s
[CV 2/5] END bootstrap=False, max_features=sqrt, min_samples_leaf=4, n_estimators=118, warm_start=False;, score=-1.007 total time=   0.6s
[CV 2/5] END bootstrap=False, max_features=1.0, min_samples_leaf=4, n_estimators=118, warm_start=True;, score=-0.988 total time=   2.0s
[CV 4/5] END bootstrap=True, max_features=1.0, min

/Users/paddy/Library/Python/3.9/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 3/5] END bootstrap=False, max_features=sqrt, min_samples_leaf=1, n_estimators=227, warm_start=True;, score=-1.102 total time=   2.5s
[CV 1/5] END bootstrap=True, max_features=log2, min_samples_leaf=1, n_estimators=118, warm_start=True;, score=-1.148 total time=   0.4s
[CV 2/5] END bootstrap=True, max_features=log2, min_samples_leaf=1, n_estimators=118, warm_start=True;, score=-1.114 total time=   0.6s
[CV 3/5] END bootstrap=True, max_features=1.0, min_samples_leaf=4, n_estimators=445, warm_start=False;, score=-0.916 total time=   7.9s
[CV 3/5] END bootstrap=True, max_features=log2, min_samples_leaf=1, n_estimators=118, warm_start=True;, score=-1.118 total time=   0.9s
[CV 4/5] END bootstrap=False, max_features=sqrt, min_samples_leaf=1, n_estimators=227, warm_start=True;, score=-1.283 total time=   3.4s
[CV 4/5] END bootstrap=True, max_features=1.0, min_samples_leaf=1, n_estimators=391, warm_start=False;, score=-1.241 total time=  12.8s
[CV 2/5] END bootstrap=True, max_features=sqrt

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=RandomForestRegressor(), n_iter=25, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_features': [1.0, 'sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'n_estimators': [10, 64, 118, 173, 227,
                                                         282, 336, 391, 445,
                                                         500],
                                        'warm_start': [True, False]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -0.9245881655849809
Best Hyperparameters: {'warm_start': False, 'n_estimators': 445, 'min_samples_leaf': 4, 'max_features': 1.0, 'bootstrap': True}
Best Model: RandomForestRegressor(min_samples_leaf=4, n_estimators=445)
Best Index: 7
CV Results: {'mean_fit_time': array([3.46544318, 1.26213403, 0.95193133, 0.7381022 , 8.40990839,
       3.17182136, 9.18089061, 8.10977077, 2.45108757, 0.90853786,
       0.41973367, 2.78096189, 8.33121119, 4.15147371, 2.50067263,
       1.30177703, 1.9174767 , 0.10821819, 3.77765555, 3.2548346 ,
       0.0671906 , 1.7550631 , 1.04036093, 5.8597599 , 2.9209156 ]), 'std_fit_time': array([1.82035916, 0.73425189, 0.4897119 , 0.36638328, 4.38232958,
       1.50152405, 4.71336799, 4.15385529, 1.20319714, 0.4265414 ,
       0.21156458, 1.3796161 , 4.62255281, 2.06707518, 1.18467947,
       0.59807871, 1.02474028, 0.05562176, 1.76909567, 1.61920184,
       0.03384954, 0.98012165, 0.52256621, 2.5575353 , 1.36228801]), 'mean_score_time': array([0.024860